In [1]:
import tensorflow as tf
import numpy as np

# Basics

## Placeholders, constants, operations and session

A `placeholder` returns a `Tensor` that may be used as a handle for feeding a value, but not evaluated directly.

It allows us to create our operations and build our computation graph, without needing the data.

In [2]:
a = tf.placeholder(tf.float32, shape=(), name= 'a')
b = tf.placeholder(tf.float32, shape=(), name='b')

A `placeholder_with_default` returns a `Tensor` that has the same type as `input`. It is a placeholder tensor that defaults to `input` if it is not fed.

In [3]:
c = tf.placeholder_with_default(5.0, shape=(), name='c')

Another type of node is a `constant`. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. This `Tensor` can be evaluated directly, it doesn't need to be fed a value.

In [4]:
d = tf.constant(4.5, tf.float32)

Placeholders do not need to be statically sized. Let’s create placeholders `e` that can take on any length and `f` that can be fed any number of rows with 3 dimensional columns

In [11]:
e = tf.placeholder(tf.float32, None)
f = tf.placeholder(tf.float32, (None, 3))

Let's define basic symbolic operations.

An `Operation` is a node in a TensorFlow `Graph` that takes zero or more `Tensor` objects as input, and produces zero or more `Tensor` objects as output.

For example `mul = tf.multiply(a, b)` creates an `Operation` of type `multiply` that takes tensors `a` and `b` as input, and produces `mul` as output.

In [12]:
mul = tf.multiply(a, b)
mul_with_default = tf.multiply(a, c)
mul_with_constant = tf.multiply(a, d)
mul_short = a * b # Shortcut
mul_with_dynamic_shape_vector = mul_short * e # Build consecutive operations in the computation graph
mul_with_dynamic_shape_matrix = a * f

A `Session` is a class for running TensorFlow operations.

A `Session` object encapsulates the environment in which `Operation` objects are executed, and `Tensor` objects are evaluated.

In [23]:
with tf.Session() as sess:
    print("Multiply two tensors: {}".format(
        sess.run(mul, feed_dict={a: 2, b: 3})))
    print("Multiply with a default placeholder: {}".format(
        sess.run(mul_with_default, feed_dict={a:2})))
    print("Multiply with a default placeholder taht is fed a value: {}".format(
        sess.run(mul_with_default, feed_dict={a:2, c:8})))
    print("Multiply with a constant: {}".format(
        sess.run(mul_with_constant, feed_dict={a:2})))
    print("Multiply two tensors using the short definition for the node: {}".format(
        sess.run(mul_short, feed_dict={a:2, b:3})))
    print("Use a dynamic shape array: {}".format(
        sess.run(mul_with_dynamic_shape_vector, feed_dict={a:2, b:3, e:[2, 3, 4]})))
    print("Use a dynamic shape array: {}".format(
        sess.run(mul_with_dynamic_shape_matrix, feed_dict={a:2, f:[[2, 3, 4], [4, 3, 2]]})))

Multiply two tensors: 6.0
Multiply with a default placeholder: 10.0
Multiply with a default placeholder taht is fed a value: 16.0
Multiply with a constant: 9.0
Multiply two tensors using the short definition for the node: 6.0
Use a dynamic shape array: [ 12.  18.  24.]
Use a dynamic shape array: [[ 4.  6.  8.]
 [ 8.  6.  4.]]


A `Session` can also be opened without the context manager (`with` statement)

In [7]:
sess = tf.Session()
print("Multiply two tensors: {}".format(sess.run(mul, feed_dict={a: 2, b: 3})))
print("Multiply two tensors: {}".format(mul.eval(feed_dict={a: 2, b: 3}, session=sess)))
sess.close()

Multiply two tensors: 6.0
Multiply two tensors: 6.0


In order to ease the work in interactive environments such as IPython notebooks or a shell, Tensorflow provides `InteractiveSession`. 

The only difference with a regular `Session` is that an `InteractiveSession` installs itself as the default session on construction. The methods `Tensor.eval()` and `Operation.run()` will use that session to run ops.

This is convenient in interactive shells and IPython notebooks, as it avoids having to pass an explicit Session object to run ops.

Note that a regular session installs itself as the default session when it is created in a `with` statement. 

In [8]:
sess = tf.InteractiveSession()

In [9]:
# We can just use 'mul.eval()' without passing 'sess'
print(mul.eval(feed_dict={a: 2, b: 3}))

6.0


In [10]:
sess.close()

## Variables

A variable maintains state in the graph across calls to `run()`. You add a variable to the graph by constructing an instance of the class `Variable`.

The `Variable()` constructor requires an initial value for the variable, which can be a `Tensor` of any type and shape. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.

If you want to change the shape of a variable later you have to use an assign Op with `validate_shape=False`.

Just like any `Tensor`, variables created with `Variable()` can be used as inputs for other Ops in the graph. Additionally, all the operators overloaded for the `Tensor` class are carried over to variables, so you can also add nodes to the graph by just doing arithmetic on variables.

In [25]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.global_variables_initializer()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

0
1
2
3


If you need to create a variable with an initial value dependent on another variable, use the other variable's `initialized_value()`. This ensures that variables are initialized in the right order.



In [27]:
# Create a variable with a random value.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
# Create another variable with the same value as 'weights'.
w2 = tf.Variable(weights.initialized_value(), name="w2")
# Create another variable with twice the value of 'weights'
w_twice = tf.Variable(weights.initialized_value() * 2.0, name="w_twice")

All variables are automatically collected in the graph where they are created. By default, the constructor adds the new variable to the graph collection `GraphKeys.GLOBAL_VARIABLES`. The convenience function `global_variables()` returns the contents of that collection.

When building a machine learning model it is often convenient to distinguish between variables holding the trainable model parameters and other variables such as a global step variable used to count training steps. To make this easier, the variable constructor supports a `trainable=<bool>` parameter. If `True`, the new variable is also added to the graph collection `GraphKeys.TRAINABLE_VARIABLES`. The convenience function `trainable_variables()` returns the contents of this collection. The various `Optimizer` classes use this collection as th£e default list of variables to optimize.

# Useful operations